# Projeto 1 - Ciência dos Dados

Nome: Andressa Silva de Oliveira

Nome: Luana Mitsudo Coelho

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'xiaomi.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo xiaomi.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

Treinamento  Valor
0               @miuirom miui 12 no rn8 xiaomi cade?      1
1  rt @dani13_f: meu cell note 8 xiaomi já tá com...      1
2  @notfakeholmes @brenomarquesof @arcanjobeni 3 ...      1
3     @juliaaacras o da xiaomi é excelente, sem mais      1
4  @rrlleal00 xiaomi mi 9 se. comprei na amazon e...      0

In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

Teste  Valor
0  vcs acham msm que a câmera d xiaomi é melhor q...      1
1  pq as autorizadas do xiaomi só são no toba do ...      1
2  🔥 em stock! smart tvs xiaomi ao preço que quer...      0
3  xiaomi poco x3 é um verdadeiro sucesso de vend...      1
4  quero refazer pq na época eu usava o iphone 7 ...      1

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Escolhemos a marca Xiaomi e consideramos, para fins de análise, que os tweets relevantes são aqueles que elogiam de alguma forma, pedem melhorias ou criticam a marca e seus produtos. Na categoria não relevantes foram classificados os tweets de propagandas de lojas e aqueles que mencionam o nome da marca mas não se encaixam nas categorias já mencionadas.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [6]:
# Limpando a base de dados para Treinamento
treinamento_limpo = cleanup(" ".join(train.Treinamento).lower())
palavras_train = treinamento_limpo.split()

In [8]:
serie_train = pd.Series(palavras_train)
serie_train

0                    @miuirom
1                        miui
2                          12
3                          no
4                         rn8
                ...          
7171                      g5s
7172                   xiaomi
7173                       mi
7174                       a2
7175    https//tco/gtqmqfuxso
Length: 7176, dtype: object

In [9]:
tabela_train_absoluta = serie_train.value_counts()
tabela_train_absoluta.head()

xiaomi    361
o         190
e         156
de        156
que       155
dtype: int64

In [10]:
tabela_train_relativa = serie_train.value_counts(True)
tabela_train_relativa.head()

xiaomi    0.050307
o         0.026477
e         0.021739
de        0.021739
que       0.021600
dtype: float64

In [12]:
# Separação de twittes em relevantes e irrelevantes
dados_R = train.loc[train.Valor == 1,:]
dados_I = train.loc[train.Valor == 0,:]

In [13]:
### Organizando a base de dados separada em relevantes e irrelevantes
dados_R_limpo = cleanup(" ".join(dados_R.Treinamento).lower())
dados_I_limpo = cleanup(" ".join(dados_I.Treinamento).lower())

In [14]:
# Separação de palavras pertencentes a twittes relevantes e irrelevantes
palavras_dados_R = dados_R_limpo.split()
palavras_dados_I = dados_I_limpo.split()

In [15]:
# Probabilidade de um twitte ser relevante ou irrelevante
prob_relev = train.Valor.value_counts(True, sort=False)
P_R = prob_relev[1]
P_I = prob_relev[0]
print(P_R)
print(P_I)

0.41456582633053224
0.5854341736694678


In [16]:
tabela_R = pd.Series(palavras_dados_R).value_counts()
tabela_R.head()

xiaomi    152
o         103
e          84
a          84
redmi      84
dtype: int64

In [17]:
tabela_I = pd.Series(palavras_dados_I).value_counts()
tabela_I.head()

xiaomi    209
o          87
que        76
de         74
e          72
dtype: int64

In [18]:
total_R = tabela_R.sum()
total_R

3572

In [19]:
total_I = tabela_I.sum()
total_I

3604

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [21]:
frase = "quero refazer pq na época eu usava o iphone 7 e agora eu tenho um xiaomi top e vai ficar bem mais legal"

In [22]:
frase = cleanup(frase.lower())

In [23]:
frase = frase.split()
frase

['quero',
 'refazer',
 'pq',
 'na',
 'época',
 'eu',
 'usava',
 'o',
 'iphone',
 '7',
 'e',
 'agora',
 'eu',
 'tenho',
 'um',
 'xiaomi',
 'top',
 'e',
 'vai',
 'ficar',
 'bem',
 'mais',
 'legal']

In [35]:
Prob_frase_R = (tabela_R.reindex(frase,fill_value = 0)+1)/(total_R + len(frase)).prod()
Prob_frase_R*P_R

vcs         0.000576
acham       0.000115
msm         0.000231
que         0.009223
a           0.009799
câmera      0.004496
d           0.000115
xiaomi      0.017639
é           0.009223
melhor      0.001383
que         0.009223
iphone?     0.000115
que         0.009223
coragy!     0.000115
não         0.004611
chega       0.000115
nem         0.001038
fodendo.    0.000115
pode        0.000346
ter         0.000576
quantos     0.000115
mp          0.000231
quiser      0.000346
fii...      0.000115
dtype: float64

In [36]:
# Limpando e criando uma coluna com cada tweet
for a in test.Teste:
    teste_limpo = cleanup(" ".join(a).lower())
    test.loc[:,'Limpos'] = teste_limpo

Teste  Valor  \
0  vcs acham msm que a câmera d xiaomi é melhor q...      1   
1  pq as autorizadas do xiaomi só são no toba do ...      1   
2  🔥 em stock! smart tvs xiaomi ao preço que quer...      0   
3  xiaomi poco x3 é um verdadeiro sucesso de vend...      1   
4  quero refazer pq na época eu usava o iphone 7 ...      1   

                                              Limpos  
0  o l d   q u e   e u   v o u   e n a l t e c e ...  
1  o l d   q u e   e u   v o u   e n a l t e c e ...  
2  o l d   q u e   e u   v o u   e n a l t e c e ...  
3  o l d   q u e   e u   v o u   e n a l t e c e ...  
4  o l d   q u e   e u   v o u   e n a l t e c e ...

In [29]:
type(test.Teste)

pandas.core.series.Series

In [34]:
# Calculando a probabilidade de cada tweet ser relevante ou irrelevante
for w in test.Limpos[0:1]:
    print(w)
    Prob_frase_R = (tabela_R.reindex(w,fill_value = 0)+1)/(total_R + len(frase)).prod()
    Prob_frase_I = (tabela_I.reindex(w,fill_value = 0)+1)/(total_I + len(frase)).prod()
    print(Prob_frase_R)
    print(Prob_frase_I)
    if (Prob_frase_R*P_R) > (Prob_frase_I*P_I):
        test.loc[:,'Classificador'] = 1
    else:
        test.loc[:,'Classificador'] = 0 

o l d   q u e   e u   v o u   e n a l t e c e r   a   m i n h a   c â m e r a   d o   m e u   x i a o m i   r e d m i   n o t e   6   p r o   h t t p s  / / t  c o / f u f d e 4 g w w h


TypeError: Index(...) must be called with a collection of some kind, 'o l d   q u e   e u   v o u   e n a l t e c e r   a   m i n h a   c â m e r a   d o   m e u   x i a o m i   r e d m i   n o t e   6   p r o   h t t p s  / / t  c o / f u f d e 4 g w w h' was passed

In [ ]:
# Tabela comparativa da classificação do modelo com a nossa
tab = pd.crosstab(test['Classificador'],test['Valor'], normalize = True)

In [ ]:
tab[0][0]

In [ ]:
# Parâmetros obtidos a partir da tabela
Porc_R_classif_R = ###
Porc_I_classif_R = 
Porc_I_classif_I = 
Porc_R_classif_I = 

print(f'Porcentagem de verdadeiros positivos = {Porc_R_classif_R}'
print(f'Porcentagem de falsos positivos = {Porc_I_classif_R}'
print(f'Porcentagem de verdadeiros negativos = {Porc_I_classif_I}'  
print(f'Porcentagem de falsos negativos = {Porc_R_classif_I}'

In [ ]:
"""
# Conta quantas vezes a palavra apareceu na base de dados
c = 0 
for p in palavras_train:
    if p == frase[i]:
        c += 1
# Calcula a probabilidade de uma palavra dada uma frase e a base de dados com todas as palavras
c = 0 
P = 1
for i in len(frase):
    for p in palavras_train:
        if p == frase[i]:
            c += 1
    P *= ((c+1/(len(frase)+len(set(palavras_train)))))

 
   
    probFraseR = 1 *tabela_train_relativa[frase[i]]
    probFraseI = 1 *tabela_train_relativa[frase[i]]
    if frase[i] not in tabela_train_relativa:
        frase[i] = 1
        """

___
### Concluindo

In [ ]:
### Conferir emojis - espaços? não deleta mesmo?

Faça um comparativo qualitativo sobre os percentuais obtidos para que possa discutir a
performance do seu classificador.

###Explique como são tratadas as mensagens com dupla negação e sarcasmo.

###Proponha um plano de expansão. Por que eles devem continuar financiando o seu
projeto? - com as melhoris sugeridas abaixo?

Discorrer por que não posso alimentar minha base de Treinamento
automaticamente usando o próprio classificador, aplicado a novos tweets.

     ###O classificador apresenta uma taxa 

Propor diferentes cenários de uso para o classificador Naive-Bayes. Pense em
outros cenários sem intersecção com este projeto.

    Dentre os cenários em que pode-se usar o classificador Naive-Bayes, têm-se:
    - Filtrar anúncios em sites de forma personalizada para cada usuário, classificando como relevantes aqueles que anunciam produtos, sites e serviços que a pessoa usualmente busca;
    - Filtrar e-mails, classificando-os como spam, principais, etc. de acordo com o conteúdo (Por exemplo: se é propaganda ou não), com o remetente (Por exemplo: se é um contato frequente ou uma conta comercial), dentre outras variàveis;
    ###- Configurar um site com filtros para mostrar produtos a partir de caraterísticas selecionadas pelo usuário (Por exemplo: Se um usuário busca pela cor rosa, produtos com essa caractrística serão dados como relevantes e aparecerão como resultado dessa busca).

Sugerir e explicar melhorias reais no classificador com indicações concretas de
como implementar (não é preciso codificar, mas indicar como fazer. Indique
material de pesquisa sobre o assunto).

    Sugestões para melhoria na performance do nosso classificador:
    - Aumentar a base de dados considerada para a construção do classificador (aumentando o valor do parâmetro "n" no   arquivo jupyter "Projeto1_Obtenção_dos_tweets.ipynb", que define a quantia mínima de mensagens capturadas para a base de treinamento e classificando-os manualmente como relevantes ou irrelevantes). A partir dessa modificação, o classificador terá mais dados nos quais se basear para fundamentar a classificação de futuros tweets, reduzindo a taxa de erros nesse processo;
    - Criar categorias intermediárias de relevância considerando a probabilidade (Para tal, deve-se criar novas variáveis que classificam cada faixa de valor, dada em probabilidade, em uma categoria. Ademais, deve-se adicionar essa classificação a uma coluna nova do tipo categórica no Dataframe). Considerando essa adaptação, a probabilidade de uma frase relevante, mesmo que em grau pequeno, ser classificada como irrelevante será reduzida.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis  OK
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis 
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

In [ ]:
"""
mt_relev = (P_R >= )|( == )
relev = ( == )|( == )
neutro = (dados['SENTIMENTO'] == 'Indiferente')
irr = (dados['SENTIMENTO'] == 'Indiferente')
mt_irr = (dados['SENTIMENTO'] == 'Indiferente')

dados.loc[mt_relev,'SENTIMENTO'] = 'Muito Relevante'
dados.loc[relev,'SENTIMENTO'] = 'Relevante'
dados.loc[neutro,'SENTIMENTO'] = 'Neutro'
dados.loc[irr,'SENTIMENTO'] = 'Irrelevante'
dados.loc[mt_irr,'SENTIMENTO'] = 'Muito Irrelevante'

dados['SENTIMENTO'] = dados['SENTIMENTO'].astype('category')
dados['SENTIMENTO'] = pd.Categorical(dados.SENTIMENTO, 
                                     categories = ['Muito Relevante', 'Relevante', 'Neutro', 'Irrelevante', 'Muito Irrelevante'], 
                                     ordered=True)
dados['SENTIMENTO'] = dados['SENTIMENTO'].cat.as_ordered()
dados['SENTIMENTO']

"""